In [21]:
import os
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

In [22]:
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = endpoint, 
  api_key= api_key,  
  api_version="2024-08-01-preview"
)

In [23]:
message_text = [
    {"role":"system","content":"Du bist ein AI Assistent der Menschen hilft Antworten zu finden."},
    {"role":"user","content":"Was ist die Telekom MMS?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,
  max_tokens=50
)

print(completion.choices[0].message.content)

Die Telekom MMS (Multimedia Messaging Service) ist ein Dienst, der von der Deutschen Telekom bereitgestellt wird und es Nutzern ermöglicht, Multimedia-Nachrichten zu versenden. Im Gegensatz zu herkömmlichen SMS, die nur Textnachrichten unterstützen


# RAG


RAG (Retrieval-Augmented Generation) ist ein Modell, das aus zwei Hauptkomponenten besteht: einem Retrieval-System und einem Generierungsmodell. Das Ziel von RAG ist es, die Qualität der Textgenerierung zu verbessern, indem es relevante und aktuelle Informationen aus einer großen Datenbank oder einem Datensatz abruft und diese Informationen zur Unterstützung des Generierungsprozesses verwendet.

Das Verfahren funktioniert in zwei Schritten:

1. Retrieval (Abruf): Zuerst sucht das Retrieval-System in einer umfangreichen Datenquelle nach Informationen, die relevant für die aktuelle Anfrage oder den Kontext sind. Diese Informationen werden als Unterstützung für die Textgenerierung ausgewählt.
2. Generation (Generierung): Anschließend verwendet das Generierungsmodell, in der Regel ein LLM, sowohl den ursprünglichen Text als auch die abgerufenen Informationen, um eine präzise und kontextuell relevante Antwort zu erzeugen.

In [24]:
prompt = """Beantworten die Fragen der Nutzenden auf der Grundlage des unten stehenden Kontexts.

context:
Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.
"""


message_text = [
    {"role":"system","content":prompt},
    {"role":"user","content":"Was ist die Telekom MMS?"},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,  
  max_tokens=50
)

print(completion.choices[0].message.content)

Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der Unternehmen, sowohl Großkonzerne als auch mittelständische, bei ihrer digitalen Transformation unterstützt. Das Unternehmen bietet verschiedene Dienstleistungen an, um die digitale Transformation in


## Laden und durchsuchen von eigenen Daten


In [25]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

2

In [26]:
# das array enthält alle texte aller Dateien

data[1]

Document(metadata={'source': 'docs/index.md'}, page_content='Personen\n\nOliver Guhr, Schulungsleiter')

### Wie können wir die Texte in kürzere Abschnitte unterteilen?

Web Demo [Chunk Visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer)

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter,MarkdownHeaderTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))
all_splits

24


[Document(metadata={'source': 'docs/mms.md'}, page_content='Telekom MMS\n\nDie Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Über 2.200 Mitarbeitende an neun Standorten in Deutschland entwickeln zukunftsfähige Geschäftsmodelle für digitale Erlebnisse. Telekom MMS bietet ein dynamisches Web- und Application-Management und sorgt mit dem hauseigenen akkreditierten Test-Center für höchste Softwarequalität, Barrierefreiheit und IT-Sicherheit.'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Das Unternehmen zählt zum Konzern Deutsche Telekom AG und tritt ebenfalls mit dem T-Logo auf. Das Unternehmen hat Standorte in Dresden, Leipzig, Jena, Berlin, Rostock, München, Stuttgart, Bonn und Hamburg. Geschichte'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Das Unter

In [28]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
chroma = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-MiniLM-L12-v2")
try:
    chroma.delete_collection("documents")
except:
    pass
collection = chroma.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=[item.page_content for item in all_splits], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[item.metadata for item in all_splits], # filter on these!
    ids=[str(id) for id in range(0,len(all_splits))], # unique for each doc
)


In [29]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Wer ist die Schulungsleiter"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

results

{'ids': [['23', '9']],
 'embeddings': None,
 'documents': [['Personen\n\nOliver Guhr, Schulungsleiter',
   'Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens. Ehemalige Geschäftsführer sind: Friedhelm Theis (1995–1998) Joachim Niemeier (1995–2005) Klaus Radermacher (1999–2002 und 2005–2007) Jens Nebendahl']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'docs/index.md'}, {'source': 'docs/mms.md'}]],
 'distances': [[8.44650650024414, 12.90152359008789]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [32]:
user = "Wer ist der Schulungsleiter?"

results = collection.query(
    query_texts=[user],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

context = "\n-\n".join(results["documents"][0])

prompt = f"""Beantworte die Fragen der Nutzenden auf der Grundlage des Kontexts.

Context:
{context}

Wenn die Informationen zur Frage nicht im Kontext enthalten sind, dann Antworte "Ich weiß es nicht."
"""

print(prompt)

message_text = [
    {"role":"system","content":prompt},
    {"role":"user","content":user},]

completion = client.chat.completions.create(
  model=deployment,
  messages = message_text,  
  max_tokens=200
)

print("----")
print(completion.choices[0].message.content)

Beantworte die Fragen der Nutzenden auf der Grundlage des Kontexts.

Context:
Personen

Oliver Guhr, Schulungsleiter
-
Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens. Ehemalige Geschäftsführer sind: Friedhelm Theis (1995–1998) Joachim Niemeier (1995–2005) Klaus Radermacher (1999–2002 und 2005–2007) Jens Nebendahl

Wenn die Informationen zur Frage nicht im Kontext enthalten sind, dann Antworte "Ich weiß es nicht."

----
Der Schulungsleiter ist Oliver Guhr.


## Nächster Schritt

Was passiert, wenn der Nutzer "Erzähle mir mehr!" im Chat eingibt? Wie können wir diese Feature implementieren?

Bonus: 

* Erstelle einen ChatBot für den EU AI Act. Welche Schritte sind dafür notwendig?

* Einstieg in [LangChain](https://www.langchain.com/). Die drei Notebooks im LangChain Ordner demonstrien wie man diesen Workflow mit Langchain implementiert.